In [1]:
# importing libraries
import re
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver import Chrome


#define function
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element_by_xpath('.//span').text
    handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responds = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responds
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text

    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet


driver = Chrome()


driver.get('https://www.twitter.com/login')
driver.maximize_window()


In [2]:
username = driver.find_element_by_xpath('//input[@name="text"]')
username.send_keys('@gk_woode')
username.send_keys(Keys.RETURN)


my_password = getpass()


password = driver.find_element_by_xpath('//input[@name="password"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)

········


In [3]:
search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys('#ghanacard')
search_input.send_keys(Keys.RETURN)


driver.find_element_by_link_text('Latest').click()


data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    for card in page_cards[-10:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
    
    scroll_attempt = 0
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(1)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            last_position = curr_position
            break
            
driver.close()

In [4]:
len(data)

577

In [5]:
with open('ghana_card.csv', 'w', newline = '', encoding = 'utf-8') as f:
    header =['Username', 'Handle', 'Timestamp', 'Comments', 'Likes', 'Retweets', 'Text']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)